In [46]:
from tkinter import N
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from bs4 import BeautifulSoup
import pandas as pd
import csv
import requests
import time
from datetime import datetime
import logging
from timeit import default_timer as timer



def get_page_of_bird(bird_name,driver):
    driver.get('https://eol.org/')
    input_eol = WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.CSS_SELECTOR,"input[name='q']")))
    input_eol.send_keys(bird_name)
    input_eol.send_keys(Keys.ENTER)
    try:
        ps = driver.find_element(by=By.CLASS_NAME, value="uk-link-reset")
        print('Eol Link ok')
        return ps.get_attribute('href')
    except Exception:
        print('Eol Link Fail')
        return None
    # return driver.current_url


def scrape_v_3_ebird(URL,bird_name):
    if URL :
        d = {}
        page = requests.get(URL)
        soup = BeautifulSoup(page.content, "html.parser")
        keys = soup.find_all("div",class_="sample-trait-key")
        values = soup.find_all(["div","a"],class_="sample-trait-val")
        des = soup.find("div",class_="desc")
        children = des.findChildren("p" , recursive=False)
        # s = ''
        # print(type(children[0]))
        # for child in children:
        #     # s += child
        #     print(child)
        #     print("---------------?")
        description = children[0].get_text()
        for i,j in zip(keys,values):
            key = i.get_text()
            value = j.get_text()
            if key in d:
                d[key] = str(d[key]) + "," + str(value)
            else:
                d[key] = value
        d2 = {}
        for i in d:
            d2[i] = [d[i]]
        # return d
            # d = {'Common Name':[b],'Kingdom':['Animalia'],'Phylum':['Chordata'],'Class':['Aves'],'Order':[a[0]],'Family':[a[1]],'Binomial Name':[c],'Description':[e],'Link':[URL]}
        d2['Link'] = [URL]
        d2['Description'] = [description]
        d2['Common Name'] = [bird_name]
        d2['Kingdom'] = ['Animalia']
        d2['Phylum']=['Chordata']
        d2['Class']=['Aves']
        # print('d2 = ',d2)
        return pd.DataFrame(d2) 
    emp_d = {} 
    # print('Eol Fail')
    return pd.DataFrame(emp_d)
    
def eol(fro,to,driver,birds):
    types_of_birds_df = pd.read_csv('assets/final_names_unq.csv',index_col='ID')
    list_of_bird_names = [ types_of_birds_df.loc[ _ ,'Common Name'] for _ in range(1,types_of_birds_df.shape[0]) ]
    for bird_name in list_of_bird_names[fro:to]:
        try:
            birds = pd.concat([birds,scrape_v_3_ebird(get_page_of_bird(bird_name,driver),bird_name)] )
            logging.debug('[+] '+ bird_name)
            print('[+] '+ bird_name)
        except Exception:
            logging.error('Unknown Exceptions for Bird-' + bird_name)
    return birds

# if __name__ == '__main__':
startTime = datetime.now()
birds = pd.DataFrame(columns=['Common Name','Kingdom','Phylum','Class','Order','Family', 'Binomial Name', 'Link','Description'])
driver = webdriver.Chrome('./chromedriver.exe')
# f = int(input('from :'))
# t = int(input('to :'))
f = 10
t = 20
ds = 'eol_'+str(f)+'_'+str(t)
logging.basicConfig(filename="assets/rescrape_eol/Log_"+ds+".log",format='%(asctime)s - %(levelname)s - %(message)s',filemode='a')
birds = eol(f,t,driver,birds)
birds.to_pickle("assets/rescrape_eol/Pick_"+ds+".pkl")
birds.to_csv("assets/rescrape_eol/csv_"+ds+".csv")
logging.info('\n\nTime taken : ',datetime.now() - startTime)
print('\n\nTime taken : ',datetime.now() - startTime) 
driver.quit()
birds.head(5)
'''
conda activate indicwiki & cd Documents\birds & python Eol_Scraper_Optim.py
<f>
<t>
'''

C:\Users\LEGION\AppData\Local\Temp\ipykernel_6724\117918712.py:88: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver.exe')


Eol Link ok
[+] Abyssinian crimsonwing
Eol Link ok
[+] Abyssinian ground hornbill
Eol Link ok
[+] Abyssinian ground thrush
Eol Link ok
[+] Abyssinian longclaw
Eol Link ok
[+] Abyssinian owl
Eol Link ok
[+] Abyssinian roller
Eol Link ok
[+] Abyssinian scimitarbill
Eol Link ok
[+] Abyssinian slaty flycatcher
Eol Link ok
[+] Abyssinian thrush
Eol Link ok
[+] Abyssinian waxbill


Time taken :  0:01:40.113277


'\nconda activate indicwiki & cd Documents\x08irds & python Eol_Scraper_Optim.py\n<f>\n<t>\n'

In [47]:
# scrape_v_3_ebird('https://eol.org/pages/1048944','Crowned Eagle')
birds.shape

(10, 32)

In [48]:
clm = list(birds.columns)
clm.sort()
clm

['Binomial Name',
 'Class',
 'Common Name',
 'Description',
 'Family',
 'Kingdom',
 'Link',
 'Order',
 'Phylum',
 'auditory system',
 'biogeographic realm',
 'body mass',
 'body shape',
 'conservation status',
 'diet includes',
 'eats',
 'ecoregion',
 'geographic distribution includes',
 'habitat',
 'hearing range',
 'hearing threshold',
 'locomotion',
 'mating system',
 'mineralized skeleton contains',
 'native range includes',
 'parental care',
 'population trend',
 'preys on',
 'social system',
 'trophic guild',
 'type specimen repository',
 'visual system']

In [49]:
birds.head(4)

,Common Name,Kingdom,Phylum,Class,Order,Family,Binomial Name,Link,Description,auditory system,...,mating system,native range includes,parental care,social system,trophic guild,hearing range,conservation status,ecoregion,eats,preys on
0,Abyssinian crimsonwing,Animalia,Chordata,Aves,NaN,NaN,NaN,https://eol.org/pages/45515637,Cryptospiza salvadorii (Abyssinian Crimson Win...,tympanic middle ear,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,Abyssinian ground hornbill,Animalia,Chordata,Aves,NaN,NaN,NaN,https://eol.org/pages/1048003,Bucorvus abyssinicus (Abyssinian Ground Hornbi...,NaN,...,monogamous,Ethiopia,provisioning,group living,invertivore,NaN,NaN,NaN,NaN,NaN
0,Abyssinian ground thrush,Animalia,Chordata,Aves,NaN,NaN,NaN,https://eol.org/pages/45515555,Geokichla piaggiae (Abyssinian Ground Thrush) ...,tympanic middle ear,...,NaN,NaN,NaN,NaN,NaN,5000 Hz,NaN,NaN,NaN,NaN
0,Abyssinian longclaw,Animalia,Chordata,Aves,NaN,NaN,NaN,https://eol.org/pages/45515069,Macronyx flavicollis (Abyssinian Longclaw) is ...,tympanic middle ear,...,NaN,NaN,NaN,NaN,NaN,NaN,near threatened,Ethiopian Montane Grasslands And Woodlands,NaN,NaN


In [42]:
scrape_v_3_ebird('https://eol.org/pages/45513608','Abyssinian roller')

,auditory system,body mass,eats,geographic distribution includes,habitat,hearing threshold,locomotion,mineralized skeleton contains,population trend,Link,Description,Common Name,Kingdom,Phylum,Class
0,tympanic middle ear,104 g,Benin,mountain,"freshwater habitat,20 dB",flight,apatite,Increasing,corneal eyes,https://eol.org/pages/45513608,Coracias abyssinicus (Abyssinian Roller) is a ...,Abyssinian roller,Animalia,Chordata,Aves
